In [4]:
import pandas as pd
import numpy as np

In [45]:
runoff = pd.read_csv('fish_river_flow.txt', sep='\t',skiprows=31, header=0)

In [46]:
runoff.head()

,5s,15s,20d,6s,14n,10s
0,USGS,2378500,2000-01-01 00:00,CST,66.0,A:[91]
1,USGS,2378500,2000-01-01 00:30,CST,66.0,A:[91]
2,USGS,2378500,2000-01-01 01:00,CST,66.0,A:[91]
3,USGS,2378500,2000-01-01 01:30,CST,66.0,A:[91]
4,USGS,2378500,2000-01-01 02:00,CST,66.0,A:[91]


In [47]:
runoff['date'] = pd.to_datetime(runoff['20d'])

In [48]:
runoff.drop(runoff[['5s','15s','20d','6s','10s']],axis=1,inplace=True)

In [49]:
runoff.head()

,14n,date
0,66.0,2000-01-01 00:00:00
1,66.0,2000-01-01 00:30:00
2,66.0,2000-01-01 01:00:00
3,66.0,2000-01-01 01:30:00
4,66.0,2000-01-01 02:00:00


In [51]:
runoff = runoff.set_index('date')

In [53]:
runoff.head()

,14n
date,
2000-01-01 00:00:00,66.0
2000-01-01 00:30:00,66.0
2000-01-01 01:00:00,66.0
2000-01-01 01:30:00,66.0
2000-01-01 02:00:00,66.0
